In [11]:
!python -m spacy download en_core_web_sm 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [1]:
pip list

Package                   Version
------------------------- -----------
annotated-types           0.5.0
anyio                     3.7.1
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.2.3
asttokens                 2.2.1
async-lru                 2.0.4
attrs                     23.1.0
Babel                     2.12.1
backcall                  0.2.0
beautifulsoup4            4.12.2
bleach                    6.0.0
blis                      0.7.10
catalogue                 2.0.9
certifi                   2023.7.22
cffi                      1.15.1
charset-normalizer        3.2.0
click                     8.1.7
cmake                     3.27.2
comm                      0.1.4
confection                0.1.1
cymem                     2.0.7
debugpy                   1.6.7.post1
decorator                 5.1.1
defusedxml                0.7.1
en-core-web-lg            3.6.0
en-core-web-sm            3.6.0
exceptiongroup            1.1.3
executing     

In [2]:
import torch
import spacy
import PyPDF2
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import sentence_transformers

In [47]:
class util:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")
        
    def batch(self, iterable, n=1):
        l = len(iterable)
        for ndx in range(0, l, n):
            yield iterable[ndx:min(ndx + n, l)]

    def cos_sim(x,y):
      return sentence_transformers.util.cos_sim(x,y)

In [48]:
class Reducer:
    def __init__(self, model_name='msmarco-MiniLM-L-6-v3', device="cpu"):
        self.device = device if torch.cuda.is_available() else "cpu"
        self.embmodel = SentenceTransformer(model_name).to(self.device)
        self.util = util()
        if self.device != "cpu":
          spacy.require_gpu()

    def reduce_small(self, text, target_embedding, num_sents=5):
        sentences = [sent.text for sent in self.util.nlp(text).sents]
        new_texts = []
        for sent in sentences:
            new_text = [s for s in sentences]
            new_text.remove(sent)
            new_texts.append(new_text)
        new_texts = [" ".join(new_text) for new_text in new_texts]
        new_text_embeddings = self.embmodel.encode(new_texts, convert_to_tensor=True, show_progress_bar=False)
        cos_scores = util.cos_sim(target_embedding, new_text_embeddings)[0]
        top_indices = torch.topk(cos_scores, k=len(new_texts))[1] #[0] isinya ada score dari ranking kemiripan kata 
        top_texts = [new_texts[i] for i in top_indices]
        top_text = top_texts[0]
        top_sentences = [sent.text for sent in self.util.nlp(top_text).sents]
        if len(top_sentences)<=num_sents:
            return top_text
        else:
            return self.reduce_small(top_text, target_embedding, num_sents)

    def reduce_large(self, big_text, target_embedding, num_sents=5):
        big_sentences = [sent.text for sent in self.util.nlp(big_text).sents]
        parts = []
        for part in self.util.batch(big_sentences, 10):
            parts.append(self.reduce_small(' '.join(part), target_embedding, num_sents))
        new = ' '.join(parts)
        if len([sent.text for sent in self.util.nlp(new).sents])<=num_sents:
            return new
        elif len(parts)==1:
            return self.reduce_small(new, target_embedding, num_sents)
        else:
            return self.reduce_large(new, target_embedding, num_sents)

    def reduce(self, huge_text, target, num_sents=5):
        target_embedding = self.embmodel.encode(target, convert_to_tensor=True, show_progress_bar=False)
        #huge_sentences = [sent.text for sent in self.self.util.nlp(huge_text).sents]
        huge_sentences = [x+"." for x in huge_text.split(". ")]
        parts = []
        for part in  self.util.batch(huge_sentences, 4):
            parts.append(" ".join(part))
        new_text_embeddings = self.embmodel.encode(parts, convert_to_tensor=True, show_progress_bar=False)
        cos_scores = util.cos_sim(target_embedding, new_text_embeddings)[0]
        top_indices = torch.topk(cos_scores, k=len(parts))
        top_texts = [parts[i] for i in top_indices[1]][:3]
        chronological = []
        for part in parts:
            if part in top_texts:
                chronological.append(part)
        reduction = self.reduce_large(" ".join(chronological), target_embedding, num_sents)
        return reduction

    def summarize(self, text, num_sents=5):
        return self.reduce(text, text, num_sents=num_sents)

    def reduce_pdf(self, pdf_path, target, num_sents=5):
        with open(pdf_path, 'rb') as f:
            pdf = PyPDF2.PdfFileReader(f)
            text = ""
            for page in pdf.pages:
                text += page.extractText()
        reduction = self.reduce(text, target, num_sents)
        return reduction

In [49]:
class QAReducer:
    def __init__(self, model_name='cross-encoder/qnli-distilroberta-base', device="cpu"):
        self.device = device if torch.cuda.is_available() else "cpu"
        self.conf_model = CrossEncoder('cross-encoder/qnli-distilroberta-base',device=0)
        self.util = util()
        if self.device != "cpu":
          spacy.require_gpu()

    def reduce_small(self, text, query, num_sents=5):
        sentences = [sent.text for sent in self.util.nlp(text).sents]
        new_texts = []
        for sent in sentences:
            new_text = [s for s in sentences]
            new_text.remove(sent)
            new_texts.append(new_text)
        new_texts = [" ".join(new_text) for new_text in new_texts]
        cos_scores = self.conf_model.predict([(query, new_text) for new_text in new_texts])
        sorted = torch.topk(torch.tensor(cos_scores), k=len(new_texts))
        top_indices = sorted[1]
       
        conf = sorted[0][0]
        top_texts = [new_texts[i] for i in top_indices]
      
        top_text = top_texts[0]
        top_sentences = [sent.text for sent in self.util.nlp(top_text).sents]
        if len(top_sentences)<=num_sents:
            return top_text, conf
        else:
            return self.reduce_small(top_text, query, num_sents)

    def reduce_large(self, big_text, query, num_sents=5):
        big_sentences = [sent.text for sent in self.util.nlp(big_text).sents]
        parts = []
        conf = 0
        for part in self.util.batch(big_sentences, 10):
            out, conf = self.reduce_small(' '.join(part), query, num_sents)
            parts.append(out)
        new = ' '.join(parts)
        if len([sent.text for sent in self.util.nlp(new).sents])<=num_sents:
            return new, conf
        elif len(parts)==1:
            return self.reduce_small(new, query, num_sents)
        else:
            return self.reduce_large(new, query, num_sents)

    def reduce(self, huge_text, query, num_sents=5):
        huge_sentences = [x+"." for x in huge_text.split(". ")]
        parts = []
        for part in  self.util.batch(huge_sentences, 4):
            parts.append(" ".join(part))
        cos_scores = self.conf_model.predict([(query, part) for part in parts])
        top_indices = torch.topk(torch.tensor(cos_scores), k=len(parts))[1]
        top_texts = [parts[i] for i in top_indices][:3]
        chronological = []
        for part in parts:
            if part in top_texts:
                chronological.append(part)
        reduction, conf = self.reduce_large(" ".join(chronological), query, num_sents)
        return reduction, conf

In [50]:
data =  [
    {
        "id_content": 1,
        "title_content": "[07042022] Analisis Target Akuisisi dan Aktivasi Merchant QRIS",
        "text_content": "===[07042022] Analisis Target Akuisisi dan Aktivasi Merchant QRIS===\n==Latar Belakang dan Tujuan==\nAnalisis ini dibuat sebagai dasar pembuatan dan desain kontrol untuk strategi akuisisi dan aktivasi merchant QRIS. Target full year 2022 adalah akuisisi 100.000 merchant QRIS, dan 1 Trilyun CASA. Dimana transaksi QRIS adalah salah satu channel untuk mencapai target CASA. Oleh karena itu perlu dilakukan analisis untuk menyusun strategi akuisisi dan aktivasi merchant QRIS, menurunkan teknis pelaksanaan di lapangan, dan desain & mekanisme kontrol pelaksanaannya.\n==Temuan Utama==\n=Simulasi Target=\nBerangkat dari target 100 ribu customer dan asumsi 1T target CASA bisa didapatkan seluruhnya dari transaksi QRIS. Skema simulasi menggunakan skema Referral yang sudah terdefinisi di dokumen ‘Target 1 Juta Nasabah’.\n=Simulasi kondisi ideal=\nDalam simulasi ini, metric yang harus dijaga untuk dapat mencapai tujuan adalah: Metric akuisisi merchant: Rasio Merchant Aktif: 20% Merchant MGM % Merchant Aktif: 30% Rata-rata Akuisisi per Merchant MGM: 5 merchant Metric aktivasi merchant: Banyaknya Transaksi per Merchant: 150 transaksi Rata-rata Nilai Transaksi: IDR 100.000 Hasil simulasi menunjukkan performa hingga 130% untuk akuisisi merchant dan 120% untuk transaksi.\n=Simulasi kondisi gradual=\nDalam simulasi ini, terdapat asumsi dimana peningkatan secara gradual jumlah merchant MGM yaitu dari 5% di April, meningkat hingga 30% di September. Asumsi metric yang lain adalah sama. Hasil simulai menunjukkan performa hanya 77% dan 73% untuk akuisisi dan transaksi, dimana keduanya masih jauh dari target.\n=Simulai kondisi realistis=\nDalam simulasi ini, ditambahkan asumsi pertumbuhan merchant aktif secara gradual mulai dari 13% (data per Maret) hingga 25% di bulan Juli. Hasil simulasi menunjukkan performa di 100% dan 98% untuk akuisisi dan transaksi.\n=Kondisi Aktual (Posisi Maret 2022)=\nSecara holistik, Rasio Merchant Aktif adalah 12.8%, Banyaknya Transaksi per Merchant adalah 30, dan Rata-rata Nilai Transaksi adalah IDR 90.886, dimana ketiganya perlu diperbaiki untuk memenuhi kriteria simulasi. Mengenai kondisi metric cabang: Rasio Merchant Aktif: Transaksi per Merchant Rata-rata Nilai Transaksi: Metric yang paling utama untuk diperbaiki adalah metric aktivasi, yaitu Rasio Merchant Aktif dan Transaksi per Merchant.\n==Rekomendasi==\nKunci keberhasilan simulasi adalah peran Merchant yang aktif sebagai perpanjangan tangan Cabang; baik untuk mengakuisisi merchant lain, dan mempromosikan penggunaan QRIS BAG. Untuk itu perlu dipastikan hubungan baik Cabang dan Merchant sehingga terjalin kerjasama yang berdampak positif terhadap bisnis Merchant dan Bank. Untuk menunjang kunci keberhasilan tersebut, perlu adanya tracking dan definisi target untuk metric tertentu kepada cabang, baik itu dalam aspek akuisisi maupun aktivasi. Perlu adanya gamifikasi sebagai sarana berkompetisi antar cabang, dan perlu diadakan reward bagi cabang yang membuktikan prestasinya. Oleh karena itu, direkomendasikan model pemantauan metric berupa kuadran capaian dengan poros-poros: Metric Akuisisi: Total Merchant, dimana pencapaian cabang dibagi berpusat pada median akuisisi cabang Metric Aktivasi: Raiso Merchant aktif, dimana pencapaian cabang dibagi berpusat pada target user aktif (20%) Target user aktif harus rutin dikaji berdasarkan realisasi capaian cabang dan kebutuhan pemenuhan target tahun 2022 Perlu adanya arahan rutin ke cabang sebagai sarana cabang untuk mengidentifikasi merchant yang perlu distimulasi untuk kembali aktif dan/atau diberikan apresiasi karena telah menjadi merchant yang loyal kepada BAG.",
        "tanggal_content": "07/24/2023",
        "author_content": "ARUM"
    },
    {
        "id_content": 2,
        "title_content": "[08042022] Database Promo Ideation",
        "text_content": "[08042022] Database Promo Ideation===\n*Berdasarkan [07042022] Analisis Target Akuisisi dan Aktivasi Merchant QRIS , rekomendasi promo yang ditargetkan adalah*\n#Metric Akuisisi: Total Merchant, dimana pencapaian cabang dibagi berpusat pada median akuisisi cabang. Metric Aktivasi: Raiso Merchant aktif, dimana pencapaian cabang dibagi berpusat pada target user aktif (20%)#\n*Dengan skema pemberian promo berdasarkan mapping kuadran ini*",
        "tanggal_content": "07/24/2023",
        "author_content": "Arum"
    },
    {
        "id_content": 3,
        "title_content": "[10032022] Analisis Transaksi Untuk Targeted Marketing",
        "text_content": "===[10032022] Analisis Transaksi Untuk Targeted Marketing===\n==Latar Belakang dan Tujuan==\n*Saat ini manajemen Lead Base kita belum optimal untuk membuat sebuah keputusan bisnis sehingga Ketika melakukan aktivitas sales ke nasabah/calon nasabah, eksekusi di lapangan masih sporadis dan tidak terfokuskan ke target pasar yang dituju Analisis ini dibuat untuk memberikan insight dalam penentuan fokus dan opsi aktivitas pre-sales dan sales ke nasabah/calon nasabah.*\n==Temuan Utama==\n*Data yang digunakan adalah data nasabah snapshot per Maret 2022, data finansial sepanjang Februari 2022, data transaksi sepanjang bulan Januari 2022.*\n==Distribusi Usia Nasabah dan Perilaku Bertransaksi==\n*Distribusi nasabah terkonsentrasi di usia produktif 25-40 tahun. Ini sejalan dengan kondisi dimana rekening artha graha banyak terdiri dari rekening payroll. Dana tersebut juga merupakan dana aktif, dalam artian perputaran dana masuk dan keluar (e.g. transaksi, transfer, pembayaran) terjadi dan lancar. Statemen tersebut juga dapat divalidasi dengan konsentrasi dana simpanan, dimana konsentrasi AUM justru tersebar di distribusi usia yang lebih tua.*\n*Artinya analisis lebih lanjut harus dilakukan di segmen usia 25-40 tahun apabila tujuan analisis adalah menemukan pola transaksi dan mendorong perilaku bertransaksi menggunakan solusi pembayaran AGI Mobile.*\n==Korelasi Usia, Jenis Pekerjaan, dan Transaksi==\n*Diatas adalah visualisasi 10 jenis pekerjaan dengan nasabah terbanyak.*\n*Terlihat bahwa konsentrasi nasabah ada di Administrasi Umum, Buruh, Pengamanan, Pegawai Swasta, dan Hospitality.*\n*Selanjutnya kita akan kembali menguatkan fokus ke segmen tersebut. *\n*Apabila mayoritas nasabah hanya melakukan transfer dana keluar BAG, hal yang perlu diketahui lagi adalah segmen nasabah mana saja yang saat ini menggunakan produk BAG untuk melakukan transaksi pembayaran atau top-up.*\n*Terlihat bahwa nasabah dengan jenis pekerjaan Administrasi Umum, Buruh, Pengamanan, Pegawai Swasta, dan Hospitality termasuk segmen nasabah yang memiliki perilaku bertransaksi pembayaran dan Top-Up e-wallet. Selain 5 segmen diatas, pelajar dan mahasiswa juga memiliki appetite yang tinggi untuk melakukan pembayaran.*\n==Kesimpulan==\n*Ada beberapa hal yang perlu digarisbawahi:*\n#Terdapat appetite untuk menggunakan platform BAG dalam melakukan pembayaran Segmen nasabah yang potensial untuk disasar adalah: Pekerja white-collar dengan income menengah kebawah (<10 juta) Pekerja blue-collar Buruh Pengamanan Segmen pelajar/mahasiswa dengan usia dibawah produktif (15-25 tahun) memiliki appetite yang tinggi dalam melakukan transaksi pembayaran walau dana yang masuk ke rekening relatif kecil daripada segmen usia produktif.#\n==Rekomendasi==\n*Untuk rekomendasi merchant, selain gerai makanan/kuliner, fokus marketing dan akuisisi merchant harus mendukung dan dirasakan menarik oleh segmen nasabah diatas. Contoh:*\n#Warmindo di sekitar lokasi pabrik/kost Bengkel modifikasi sepeda motor Distributor sparepart dan aksesoris sepeda motor Distribusi sembako#\nPerlu riset dan survey lebih lanjut ke segmen nasabah tersebut untuk mencari tahu lebih jauh mengenai preferensi transaksi mereka. Perlu riset dan survey lebih lanjut terhadap kompetitor: OVO dan LINKAJA. Terutama sub-produk apa dari kompetitor yang dipakai oleh nasabah BAG.#\n==Action Items==\n*[ ] [PIC: Agnes & Dian] Penyusunan program acquisition & promo untuk:*\n#Pulsa & paket data, Games —> untuk segment pelajar & mahasiswa —> Mobile Legend, Free Fire, Apex Legend Bengkel Modifikasi & Aksesori Motor —> untuk segment buruh dan pekerja Sembako —> untuk karyawan group dan all existing nasabah Warmindo (warung indomie) —> untuk segment buruh dan pekerja % Agi Mobile User New Customer/Conversion Feebased income Cost/Burn rate#\n*[x] [PIC: Bu Erika] Penyusunan naskah survey untuk riset lebih lanjut tentang use-case mobile banking/payment di segmen tersebut; Target: Senin 14 Maret 2022 [ ] [PIC: Bu Erika + Indri] Eksekusi survey untuk existing nasabah atau pengguna didalam group AG Target: 361 data points [ ] [PIC: Agnes] Eksekusi survey untuk target segment diluar group Target: 361 data points",
        "tanggal_content": "07/24/2023",
        "author_content": "Administrator"
    }    
    ]

combined_text = '\n\n'.join(item['text_content'] for item in data)

In [51]:
print(combined_text)

===[07042022] Analisis Target Akuisisi dan Aktivasi Merchant QRIS===
==Latar Belakang dan Tujuan==
Analisis ini dibuat sebagai dasar pembuatan dan desain kontrol untuk strategi akuisisi dan aktivasi merchant QRIS. Target full year 2022 adalah akuisisi 100.000 merchant QRIS, dan 1 Trilyun CASA. Dimana transaksi QRIS adalah salah satu channel untuk mencapai target CASA. Oleh karena itu perlu dilakukan analisis untuk menyusun strategi akuisisi dan aktivasi merchant QRIS, menurunkan teknis pelaksanaan di lapangan, dan desain & mekanisme kontrol pelaksanaannya.
==Temuan Utama==
=Simulasi Target=
Berangkat dari target 100 ribu customer dan asumsi 1T target CASA bisa didapatkan seluruhnya dari transaksi QRIS. Skema simulasi menggunakan skema Referral yang sudah terdefinisi di dokumen ‘Target 1 Juta Nasabah’.
=Simulasi kondisi ideal=
Dalam simulasi ini, metric yang harus dijaga untuk dapat mencapai tujuan adalah: Metric akuisisi merchant: Rasio Merchant Aktif: 20% Merchant MGM % Merchant Aktif

In [59]:
modelBaca = QAReducer()
answer= modelBaca.reduce(combined_text, "target akuisisi tahun 2022", num_sents=5)

print(answer)

('Analisis Target Akuisisi dan Aktivasi Merchant QRIS===\n==Latar Belakang dan Tujuan==\nAnalisis ini dibuat sebagai dasar pembuatan dan desain kontrol untuk strategi akuisisi dan aktivasi merchant QRIS. ==Temuan Utama==\n=Simulasi Target=\nBerangkat dari target 100 ribu customer dan asumsi 1T target CASA bisa didapatkan seluruhnya dari transaksi QRIS. Target user aktif harus rutin dikaji berdasarkan realisasi capaian cabang dan kebutuhan pemenuhan target tahun 2022 [07042022] Analisis Transaksi Untuk Targeted Marketing===\n==Latar Belakang dan Tujuan==\n*Saat ini manajemen Lead Base kita belum optimal untuk membuat sebuah keputusan bisnis sehingga Ketika melakukan aktivitas sales ke nasabah/calon nasabah, eksekusi di lapangan masih sporadis dan tidak terfokuskan ke target pasar yang dituju Analisis ini dibuat untuk memberikan insight dalam penentuan fokus dan opsi aktivitas pre-sales dan sales ke nasabah/calon nasabah.', tensor(0.9984))
